# Parameters and constructions

In [2]:
p = 2
m = 1
q = p^m
K.<alpha> = GF(q)
N = 12
L_N.<a> = GF(q^N)
P = L_N._modulus

In [3]:
x = ['x'+str(k) for k in range(N)]
M = PolynomialRing(L_N,N,x)
I = M.ideal([M(x[i])^q - M(x[i]) for i in range(N)])
Q = M.quotient(I)

R.<X> = PolynomialRing(M)
S = R.quotient(R(P))

variable = sum([M(x[i])*X^i for i in range(N)])

def HFE_representation_polynome(f):
    """
    Param: f, a polynomial in L_N[X] 
    Return: (p0,p1,..,pn), polynomials in Fq[x0,..,xN-1] such that 
            f(x0,..,xn) = (p0(x0,..,xn),.., pn(x0,..xn))
    """
    f_prime = R(f) # we look f in the ring L_N[x1,...,xN][X]

    f_prime = S(f_prime(variable))
    g =M(f_prime.lift()(a))
    
    f = 0
    for i in g.monomials():
        f+= (X^L_N(g.monomial_coefficient(i)).log(a)*i)
    # we evaluate f on x0 + x1*X + .. + xN*X^N 
    # we reduce by the modulus 
    # which give us a polynomials of degree < N
    p_list = [Q(pi).lift() for pi in list(S(f).lift())]
    
    return p_list

# HFE Cryptosystem

In [4]:
def HFE_function(d, number_coeff):
    """
    Param: d, degree of f 
            number_coeff, number of coefficient nonzero of f
    Sortie: f, polynomial which verify the HFE conditions
    """
    
    coeff = [L_N.random_element() for i in range(number_coeff)]
    f = 0
    lim = abs(floor(log(d,q)-1))
    for i in range(len(coeff)-2):
        theta = randint(0,lim)
        phi = randint(0,lim)
        f += coeff[i] * X^(q^theta + q^phi)
        
    mu_0 = L_N.random_element()
    f += mu_0 + coeff[-1] * X^d 
    return f

In [5]:
def HFE_generates_keys(d, number_coeff):
    f = HFE_function(d, number_coeff)
    
    T.<Y> = PolynomialRing(L_N)

    s = R(T.random_element(1))
    t = R(T.random_element(1))
    
    p_list = HFE_representation_polynome(s(f(t)))
    private_key = (L_N,f,s,t)
    public_key = (K, p_list)
    return public_key, private_key 

In [7]:
def HFE_encryption(m, public_key):
    """
    Param: m, an element of (F_q)^N
    Return: c, c = HFE(m)
    """
    K, p_list = public_key
    m = tuple(m)
    return [p(m) for p in p_list]

In [8]:
def HFE_decryption(y, private_key):
    L_N, f, s, t = private_key
    a = L_N([0,1])
    y = sum([y[i]*a**i for i in range(N)])

    list_s = list(s)
    s_inv = list_s[1]**(-1) * (X - list_s[0])
    list_t = list(t)
    t_inv = list_t[1]**(-1) * (X - list_t[0])
        
    ft = s_inv(y)
    
    v = tuple((L_N^N).zero())
    
    roots = (f-ft).roots(multiplicities = False)
    m = [L_N.vector_space(map = false)((t_inv(r))(v)) for r in roots]
   
    return m

## Example and verification

In [9]:
t = cputime()
public_key, private_key = HFE_generates_keys(12, 4)
cputime(t)

3.5671789999999994

In [10]:
private_key

(Finite Field in a of size 2^12,
 ((a^10 + a^9 + a^5 + a^4 + a^3 + a^2 + a))*X^12 + ((a^10 + a^3 + a + 1))*X^6 + ((a^10 + a^9 + a^7 + a^6 + a^5))*X^5 + (a^10 + a^9 + a^8 + a^7 + a^6 + a^4 + a^3),
 ((a^10 + a^8 + a^7 + a^2 + a + 1))*X + (a^10 + a^5 + a^3 + a^2 + a + 1),
 ((a^9 + a^8 + a^7 + a^5 + a^3 + a^2 + a + 1))*X + (a^8 + a^7 + a^5 + a^4 + a^2))

In [11]:
v = L_N.vector_space(map=False)(L_N.random_element())
p = HFE_encryption(v,public_key)
m = HFE_decryption(p, private_key)
m

[(0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1), (1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0)]

In [12]:
v

(1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0)

# Cryptanalysis with Gröbner basis

In [13]:
def solve_sys(S,B):
    def solve_sys_rec(S,expens,i):
        expension = expens.copy()
        if i == 0:
            return expension
        Scopy = S.copy()
        f = Scopy[i-1]
        if f == 0:
            return solve_sys_rec(Scopy,expension,i-1)
        f = f.univariate_polynomial()
        root = f.roots(multiplicities = False)
        key = f.variable_name()
        if (len(root) == 1):
            r = root[0]
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            return solve_sys_rec(new_S,expension,i-1)
        res = []
        for r in root:
            new_S = []
            expension[B(key)] = r
            for s in Scopy:
                new_S += [s.subs(expension)] 
            res += [solve_sys_rec(new_S,expension,i-1)]        
        return res
    
    expension = {B(xi):B(xi) for xi in x}
    l = solve_sys_rec(S, expension, len(S))
    return (l)

In [18]:
def HFE_cryptanalysisI(public_key,p):
    K,Sys = public_key
    N = len(Sys)
    x = Sys[0].variables()
    B = PolynomialRing(K,N,x, order = 'lex')
    Sys = [Sys[i] - p[i] for i in range(N)] + [B(xi)^2-B(xi) for xi in x]
    IS = Ideal(B,Sys)
    Sols = IS.groebner_basis()
    return solve_sys(Sols,B)

Give a recursive list of dict, we could give a better representation with vector

In [19]:
HFE_cryptanalysisI(public_key,p)

[{x0: 1,
  x1: 1,
  x2: 1,
  x3: 1,
  x4: 0,
  x5: 1,
  x6: 1,
  x7: 0,
  x8: 1,
  x9: 1,
  x10: 0,
  x11: 0},
 {x0: 0,
  x1: 1,
  x2: 0,
  x3: 1,
  x4: 1,
  x5: 1,
  x6: 1,
  x7: 1,
  x8: 1,
  x9: 0,
  x10: 1,
  x11: 1}]

In [20]:
m

[(0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1), (1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0)]

In [21]:
v

(1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0)